In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from bert_sklearn import BertClassifier

from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV

In [4]:
data = pd.read_csv('data/train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
pipeline = Pipeline(
    [
        ("clf", BertClassifier(max_seq_length=64, train_batch_size=16)),
    ]
)
pipeline

Building sklearn text classifier...


Pipeline(steps=[('clf',
                 BertClassifier(max_seq_length=64, train_batch_size=16))])

In [6]:
param_grid = {
    'clf__epochs': [3, 4], 
    'clf__learning_rate': [2e-5, 3e-5, 5e-5]
}

In [7]:
grid = GridSearchCV(
    pipeline, 
    param_grid, 
    scoring='f1',
    cv = 10
)
grid

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('clf',
                                        BertClassifier(max_seq_length=64,
                                                       train_batch_size=16))]),
             param_grid={'clf__epochs': [3, 4],
                         'clf__learning_rate': [2e-05, 3e-05, 5e-05]},
             scoring='f1')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["target"])

In [ ]:
grid.fit(X_train, y_train)

Building sklearn text classifier...
Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 4625, validation data size: 513


Training  :   0%|                                                                                                                                                   | 0/290 [00:00<?, ?it/s]/home/renanlf/.local/lib/python3.10/site-packages/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
Training  :   0%|▍                                                                                                                            | 1/290 [00:30<2:27:08, 30.55s/it, loss=0.676]

In [2]:
y_predicted = grid.predict(X_test)

NameError: name 'grid' is not defined

In [86]:
f1_score(y_test, y_predicted)

0.7501647989452868

In [87]:
grid.best_score_, grid.best_params_

(0.7464505974870467, {'clf__kernel': 'linear', 'vect__analyzer': 'word'})

In [88]:
test_data = pd.read_csv('data/test.csv')

In [89]:
target = grid.predict(test_data["text"])

In [90]:
output = pd.DataFrame(data = {
    "id": test_data["id"],
    "target": target
})
output.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [91]:
output.to_csv("submission.csv", index=False)

In [92]:
!kaggle competitions submit -c nlp-getting-started -f "submission.csv" -m "Submission using simple SVC and TF-IDF vectorizer with grid search params (linear, word)"

100%|██████████████████████████████████████| 22.2k/22.2k [00:01<00:00, 21.8kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets